In [4]:
# ----------------------------------------------------------
# 📌 Step 1: Install Required Libraries
# ----------------------------------------------------------
!pip install tensorflow keras numpy matplotlib scikit-learn opencv-python pillow


# ----------------------------------------------------------
# 📌 Step 2: Import Libraries
# ----------------------------------------------------------
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
import matplotlib.pyplot as plt
import numpy as np
import cv2


# ----------------------------------------------------------
# 📌 Step 3: Load Dataset
# Folder structure:
# dataset/
#     clean/
#     polluted/
# ----------------------------------------------------------
train_data = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2
)

train_ds = train_data.flow_from_directory(
    'dataset/',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_ds = train_data.flow_from_directory(
    'dataset/',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


# ----------------------------------------------------------
# 📌 Step 4: Build CNN Model
# ----------------------------------------------------------
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


# ----------------------------------------------------------
# 📌 Step 5: Train the Model
# ----------------------------------------------------------
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


# ----------------------------------------------------------
# 📌 Step 6: Evaluate Accuracy
# ----------------------------------------------------------
loss, accuracy = model.evaluate(val_ds)
print("Validation Accuracy:", accuracy)


# ----------------------------------------------------------
# 📌 Step 7: Plot Accuracy & Loss Graphs
# ----------------------------------------------------------
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.legend()
plt.show()


# ----------------------------------------------------------
# 📌 Step 8: Save the Model
# ----------------------------------------------------------
model.save("EcoVisionAI_model.h5")
print("Model saved successfully as EcoVisionAI_model.h5")


# ----------------------------------------------------------
# 📌 Step 9: Predict on a Single Image
# Place your test image in a folder called "test_img"
# Example: test_img/sample.jpg
# ----------------------------------------------------------
def predict_image(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224,224))
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)[0][0]

    if prediction > 0.5:
        print("Prediction: 🌫️ Polluted Area")
    else:
        print("Prediction: 🌿 Clean Area")

# Example call (change image name):
# predict_image("test_img/sample.jpg")


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'dataset/'